# Prices and costs analyses
Routines to evaluate and visualize price and costs assumptions used as input for POMMES simulations

## Package imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

## Define data sets and global variables
* Define some model configuration defining which input to use
* Determine paths and file names for inputs and outputs to the script

In [ ]:
# Model configuration in terms of prices and costs
fuel_cost_pathway = "NZE"
emissions_cost_pathway = "long-term"
inflation_rate = 0.02

# inputs and outputs
path_folder = "./model_inputs/pommesinvest/"
file_name_fuel_costs_ts = f"costs_fuel_{fuel_cost_pathway}_nominal_indexed_ts.csv"
file_name_emissions_costs_ts = f"costs_emissions_{emissions_cost_pathway}_nominal_indexed_ts.csv"

path_data_out = "./data_out/"
path_plots = "./plots/"
output_file = "fuel_and_emissions_costs"

# global variables
FUELS = {
    #"biomass": "#15b01a",
    #"uranium": "#e50000",
    #"lignite": "#7f2b0a",
    "hardcoal": "#000000",
    "natgas": "#ffd966",
    "hydrogen": "#6fa8dc",
    #"mixedfuels": "#a57e52",
    #"otherfossil": "#d8dcd6",
    #"waste": "#c04e01",
    "oil": "#aaa662",
}
plt.rcParams.update({'font.size': 14})

## Read in data
* Read in **_nominal_** fuel and emissions prices data.
* Convert to **_real_** terms using the assumed inflation rate of 2% p.a.

In [ ]:
fuel_costs_nominal = pd.read_csv(f"{path_folder}{file_name_fuel_costs_ts}", index_col=0)
emissions_costs_nominal = pd.read_csv(f"{path_folder}{file_name_emissions_costs_ts}", index_col=0)

In [ ]:
# Filter values for DE only
fuel_costs_nominal = fuel_costs_nominal[
    [col for col in fuel_costs_nominal.columns if "DE_" in col]
]
emissions_costs_nominal = emissions_costs_nominal[
    [col for col in emissions_costs_nominal.columns if "DE_" in col]
]

fuel_costs_nominal.index = fuel_costs_nominal.index.str[:4].astype(int)
emissions_costs_nominal.index = emissions_costs_nominal.index.str[:4].astype(int)

# Convert from nominal to real terms
fuel_costs_real = fuel_costs_nominal.div(
    (1 + inflation_rate) ** (fuel_costs_nominal.index - 2020), axis=0
)
emissions_costs_real = emissions_costs_nominal.div(
    (1 + inflation_rate) ** (emissions_costs_nominal.index - 2020), axis=0
)

In [ ]:
# Rename
fuel_costs_real.rename(
    columns={col: col.rsplit("_")[-1] for col in fuel_costs_real.columns},
    inplace=True
)

# Emissions costs are the same for all energy carriers
emissions_costs_real = emissions_costs_real.loc[2020:2050, "DE_source_hardcoal"]
emissions_costs_real.name = "co2_price"

In [ ]:
# create a combined plot
fig, ax = plt.subplots(figsize=(12, 5))
fuel_costs_real = fuel_costs_real.loc[2020:2050, [fuel for fuel in FUELS]]
fuel_costs_real.to_csv(f"{path_data_out}{output_file}.csv")

_ = fuel_costs_real.loc[2020:2045].plot(kind="line", marker="o", ax=ax, color=FUELS)

ax2 = ax.twinx()

_ = emissions_costs_real.loc[2020:2045].plot(kind="line", marker="o", ax=ax2, color="#898989")

_ = ax.set_xlabel("year")
_ = ax.set_ylabel("price in €/MWh_th")
_ = ax2.set_ylabel("price in €/t CO_2")

_ = ax.legend(bbox_to_anchor=[1.35, 1.05])
_ = ax2.legend(bbox_to_anchor=[1.35, 0.7])
_ = ax.set_ylim([0, 200])
_ = ax2.set_ylim([0, 400])
#_ = ax.set_xticks(range(2020, int(max(fuel_costs_real.index + 1)), 1))
_ = ax.autoscale(enable=True, axis='x', tight=True)

ax.grid(color="#dedede")
plt.tight_layout()
plt.savefig(f"{path_plots}{output_file}.png", dpi=300)
plt.show()
plt.close()